# Catalogue of Somatic Mutations In Cancer (COSMIC) - Mutations

Created by: Charles Dai <br>
Credit to: Moshe Silverstein

Data Source: https://cancer.sanger.ac.uk/cosmic/download

In [1]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [2]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [3]:
# from clustergrammer_widget import *
# net = Network(clustergrammer_widget)

In [4]:
%load_ext autoreload
%autoreload 2

### Notebook Information

In [5]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

This notebook was run on: 2020-06-29 
Python version: 3.8.0 (default, Oct 28 2019, 16:14:01) 
[GCC 8.3.0]


# Initialization

### Load Mapping Dictionaries

In [6]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

Gathering sources: 100%|██████████| 3/3 [00:08<00:00,  2.96s/it]


### Output Path

In [8]:
output_name = 'cosmic_mutations'

path = 'Output/COSMIC-Mutations'
if not os.path.exists(path):
    os.makedirs(path)

In [9]:
%%appyter hide_code
{% do SectionField(
    name='data',
    title='Load Data',
    subtitle='Upload Files from the Catalogue of Somatic Mutations In Cancer',
) %}

# Load Data

In [39]:
%%appyter code_exec

df = pd.read_csv({{FileField(
    constraint='.*\.tsv.gz$',
    name='mutation_data', 
    label='Mutation Data (tsv.gz)', 
    default='Input/COSMIC/CosmicCompleteTargetedScreensMutantExport.tsv.gz',
    section='data')
}}, 
    sep='\t', usecols=['Gene name', 'Sample name'], index_col=0)

```python
df = pd.read_csv('Input/COSMIC/CosmicCompleteTargetedScreensMutantExport.tsv.gz',
    sep='\t', usecols=['Gene name', 'Sample name'], index_col=0, nrows=1000)
```

In [40]:
df.head()

,Sample name
Gene name,
CDKN2A_ENST00000579755,1165234
POLD1_ENST00000599857,P-0013227-T01-IM5
DNM2,M18
DNM2,61
CUX1_ENST00000547394,B36


In [41]:
df.shape

(1000, 1)

# Pre-process Data

## Group Mutations of Same Gene Together

In [43]:
df.index = df.index.map(lambda s: s.split('_')[0])

In [44]:
df.index.name = 'Gene Symbol'
df.columns = ['Sample Name']
df.head()

,Sample Name
Gene Symbol,
CDKN2A,1165234
POLD1,P-0013227-T01-IM5
DNM2,M18
DNM2,61
CUX1,B36


# Filter Data

## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [45]:
df = uf.map_symbols(df, symbol_lookup, remove_duplicates=True)
df.shape

100%|██████████| 1000/1000 [00:00<00:00, 103816.84it/s]


(987, 1)

# Analyze Data

## Create Binary Matrix

In [ ]:
binary_matrix = uf.binary_matrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.save_data(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

## Create Gene List

In [ ]:
gene_list = uf.gene_list(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.save_data(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

## Create Attribute List

In [ ]:
attribute_list = uf.attribute_list(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.save_data(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

## Create Gene and Attribute Set Libraries

In [ ]:
uf.save_setlib(binary_matrix, 'gene', 'up', path, output_name + '_gene_up_set')

In [ ]:
uf.save_setlib(binary_matrix, 'attribute', 'up', path, 
                           output_name + '_attribute_up_set')

## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.similarity_matrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.save_data(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

In [ ]:
# net.load_df(attribute_similarity_matrix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.similarity_matrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.save_data(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene-Attribute Edge List

In [ ]:
edge_list = uf.edge_list(binary_matrix)
uf.save_data(edge_list, path, output_name + '_edge_list', 
        ext='tsv', compression='gzip')

# Create Downloadable Save File

In [ ]:
uf.archive(path)

### Link to download output files: [click here](./output_archive.zip)